Questão 1:
O fundo mais lucrativo da principal empresa de gestão de ativos de
Gotham city segue umaregra simples: o lucro é o juros composto 
calculado com a taxa de juros SELIC. Seu desafio é criar um programa que
calcule uma série que contenha o lucro total, dado um valor inicial de
capital em um determinado período de tempo (data de início, data de
término) com diferentes frequências. Sua solução deve seguir as 
restrições abaixo:
- A data de início deve ser maior ou igual a 1995-01-01
- A data de término deve ser posterior à data de início
- A frequência deve ser dia, mês ou ano

# Importação das bibliotecas

In [1]:
from datetime import datetime, date
import matplotlib.pyplot as plt
import numpy as np
from bcb import sgs

# Coletando dados do usuário

In [13]:
data = {}

capital = float(input('Digite o capital investido:\t'))
frequencia = str(input('Digite a frequência investido(Y, ME, D):\t'))
data['inicial'] = str(input('Digite a data inicial após 1995/01/01  no formato YYYY/MM/DD:\t'))
data['final'] = str(input('Digite a data final no seguinte formato YYYY/MM/DD:\t'))

# Tratamento de dados

In [7]:
data['inicial'] = datetime.strptime(data_inicial, '%Y/%m/%d').date()
data['final'] = datetime.strptime(data_final, '%Y/%m/%d').date()

data

{'inicial': datetime.date(1996, 1, 1), 'final': datetime.date(2023, 12, 31)}

# Pegar dados da SELIC do Banco Central

In [9]:
taxa_selic = sgs.get({'selic':11}, start = data['inicial'], end = data['final'])

taxa_selic

,selic
Date,
1996-01-02,0.120000
1996-01-03,0.119667
1996-01-04,0.114667
1996-01-05,0.113667
1996-01-08,0.113667
...,...
2023-12-22,0.043739
2023-12-26,0.043739
2023-12-27,0.043739


# Calcular retorno no período

In [10]:
taxa_selic = taxa_selic / 100

taxa_selic

,selic
Date,
1996-01-02,0.001200
1996-01-03,0.001197
1996-01-04,0.001147
1996-01-05,0.001137
1996-01-08,0.001137
...,...
2023-12-22,0.000437
2023-12-26,0.000437
2023-12-27,0.000437


In [14]:
capital_acumulado = capital * (1 + taxa_selic['selic']).cumprod() - 1

capital_acumulado

Date
1996-01-02     1000.200000
1996-01-03     1001.398106
1996-01-04     1002.547526
1996-01-05     1003.688228
1996-01-08     1004.830227
                  ...     
2023-12-22    39888.711891
2023-12-26    39906.159252
2023-12-27    39923.614244
2023-12-28    39941.076872
2023-12-29    39958.547137
Name: selic, Length: 7029, dtype: float64

In [15]:
capital_com_frequencia = capital_acumulado.resample(frequencia).last()
# Observe que nesse caso, pegamos os dados de fechamento,
# mas poderíamos, sem nenhum problema, pegar a média, por exemplo 
# com um .mean() ao final do código, no lugar de ,last()

capital_com_frequencia


# Ao fim da aula, ele fala sobre utilizar essa mesma lógica para a criação de um dashboard
#, ao que eu observei ele utiliza as seguintes bibliotecas para fazer dashboards:
# dash(import dcc, html, Input, Output, dash_table), dash_bootstrap_components
# e plotly.express(essa eu acredito que seja pra visualização gráfica)

Date
1996-01-31     1024.764965
1996-02-29     1048.885472
1996-03-31     1072.201557
1996-04-30     1094.395862
1996-05-31     1116.443476
                  ...     
2023-08-31    38482.736539
2023-09-30    38857.145487
2023-10-31    39244.781610
2023-11-30    39604.268192
2023-12-31    39958.547137
Freq: ME, Name: selic, Length: 336, dtype: float64

Questão 2

Depois de desenvolver sua solução, responda a esta pergunta:
Qual foi o período mais lucrativo desde o início de 2000-01-01 2022-03-31? Ou seja, se você tivesse que investir um valor C de capital por 500 dias, qual teria sido o período mais lucrativo desde o início de 2000 até o final de março de 2022? Sua resposta deve ser as datas de início e término do período encontrado. 

# Filtrar dados da selic no período solicitado

In [21]:
data_inicial = date(2000, 1, 1)
data_final = date(2022, 12, 31)

selic_questao2 = sgs.get({'selic':11}, start = data_inicial, end = data_final)/100

selic_questao2


,selic
Date,
2000-01-03,0.000692
2000-01-04,0.000692
2000-01-05,0.000692
2000-01-06,0.000693
2000-01-07,0.000693
...,...
2022-12-26,0.000508
2022-12-27,0.000508
2022-12-28,0.000508


# Calcular a rentabilidade das janelas de 500 dias

In [22]:
janela_500_dias = ((1 + selic_questao2).rolling(window = 500).apply(np.prod) -1)
janela_500_dias

# Uma sugestão legal: eu acredito que ficaria bem mais intuitivo se
# os dados representasse o dia que deveria começar o investimento
# e não o fim deles, o que pode ser resolvido de maneira simples 
# deslocando os dados dessa tabela. Tarefa essa que eu deixo para 
# o Kaio do futuro

,selic
Date,
2000-01-03,NaN
2000-01-04,NaN
2000-01-05,NaN
2000-01-06,NaN
2000-01-07,NaN
...,...
2022-12-26,0.171421
2022-12-27,0.171929
2022-12-28,0.172436


In [23]:
janela_500_dias = janela_500_dias.reset_index()

janela_500_dias

,Date,selic
0,2000-01-03,NaN
1,2000-01-04,NaN
2,2000-01-05,NaN
3,2000-01-06,NaN
4,2000-01-07,NaN
...,...,...
5772,2022-12-26,0.171421
5773,2022-12-27,0.171929
5774,2022-12-28,0.172436
5775,2022-12-29,0.172944


# Criar range de datas na tabela

In [24]:
# desloca as datas em 500 dias, para bater com o período esperado
janela_500_dias['Data_inicial'] = janela_500_dias['Date'].shift(500)
janela_500_dias

,Date,selic,Data_inicial
0,2000-01-03,NaN,NaT
1,2000-01-04,NaN,NaT
2,2000-01-05,NaN,NaT
3,2000-01-06,NaN,NaT
4,2000-01-07,NaN,NaT
...,...,...,...
5772,2022-12-26,0.171421,2020-12-29
5773,2022-12-27,0.171929,2020-12-30
5774,2022-12-28,0.172436,2020-12-31
5775,2022-12-29,0.172944,2021-01-04


In [28]:
janela_500_dias = janela_500_dias.dropna()
janela_500_dias

,Date,selic,Data_inicial
500,2002-01-02,0.377688,2000-01-03
501,2002-01-03,0.377689,2000-01-04
502,2002-01-04,0.377689,2000-01-05
503,2002-01-07,0.377688,2000-01-06
504,2002-01-08,0.377687,2000-01-07
...,...,...,...
5772,2022-12-26,0.171421,2020-12-29
5773,2022-12-27,0.171929,2020-12-30
5774,2022-12-28,0.172436,2020-12-31
5775,2022-12-29,0.172944,2021-01-04


In [29]:
janela_500_dias.columns = ['Data_final', 'retorno_selic_500dias', 'data_inicial']
janela_500_dias

,Data_final,retorno_selic_500dias,data_inicial
500,2002-01-02,0.377688,2000-01-03
501,2002-01-03,0.377689,2000-01-04
502,2002-01-04,0.377689,2000-01-05
503,2002-01-07,0.377688,2000-01-06
504,2002-01-08,0.377687,2000-01-07
...,...,...,...
5772,2022-12-26,0.171421,2020-12-29
5773,2022-12-27,0.171929,2020-12-30
5774,2022-12-28,0.172436,2020-12-31
5775,2022-12-29,0.172944,2021-01-04


# Pegar o maior retorno da tabela

In [30]:
maior_retorno = janela_500_dias['retorno_selic_500dias'].max()
maior_retorno

np.float64(0.4669665013674005)

In [31]:
gabarito = janela_500_dias[janela_500_dias['retorno_selic_500dias'] == maior_retorno]

gabarito

,Data_final,retorno_selic_500dias,data_inicial
956,2003-10-22,0.466967,2001-10-26
